In [3]:
import pandas as pd

In [4]:
# HoBo
df_oo = pd.read_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hobosd.csv')
# test = pd.read_csv('/content/drive/MyDrive/ch5. 소득예측/test_oom.csv')

In [5]:
# 함수 만들기
def label_encoder(df) :
  # 라이브러리 불러오기
  from sklearn.preprocessing import LabelEncoder
  # 인코딩할 칼럼 추출
  df_categorical = df.select_dtypes(include=['object']).columns
  # 식별자 ID 제외하기
  df_categorical = df_categorical.drop('ID')
  # 라벨 인코딩 적용
  label_encoder = LabelEncoder()
  df[df_categorical] = df[df_categorical].apply(label_encoder.fit_transform)

In [6]:
label_encoder(df_oo)

In [7]:
# # # features와 label 분리 작업 + 연령대로 머신러닝
X_features = df_oo.loc[:,(df_oo.columns != 'ID') & (df_oo.columns != 'Income')]
y_label = df_oo['Income']

In [8]:
# train_ test분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_features, y_label, test_size = 0.2, random_state = 42)

In [9]:
# 평가지표
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, median_absolute_error
import numpy as np

def evaluation(y_test, y_pred) :
  mae = mean_absolute_error(y_test, y_pred)
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  # MdAPE를 잘 쓰곤 함. 직관적이여서(외부공유)
  mdape = np.median(np.abs((y_test - y_pred) / y_test)) * 100
  mape = mean_absolute_percentage_error(y_test,y_pred)
  print('rmse : {}, mae : {}, mdape : {}, mape : {}'.format(rmse, mae, mdape, mape ))

# Catboost

In [15]:
# !pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.8 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostRegressor
# 모델
cat_model = CatBoostRegressor()
# 학습
cat_model.fit(X_train, y_train)
# 예측
ct_pred = cat_model.predict(X_test)
# 평가
evaluation(y_test, ct_pred)

In [17]:
feature_importance = cat_model.get_feature_importance(type='FeatureImportance')

In [18]:
feature_importance

array([ 6.62401222,  3.20657782,  5.36244241,  1.6025567 ,  5.2995102 ,
       16.84897071, 24.52732681,  1.50599212,  1.19795914,  2.23310729,
        2.38920697,  2.24899331,  1.5543465 ,  1.04208724,  0.93345049,
        0.47979642,  4.1983941 ,  3.91920659,  0.31698397,  0.80263357,
        2.56963029,  2.63594527,  0.52733209,  1.98072489,  0.54311316,
        1.27468761,  1.91660826,  0.81596385,  0.82948012,  0.17943405,
        0.1630001 ,  0.27052571])

In [19]:
list_col = feature_importance
index_x = X_train.columns

# 딕셔너리 생성
data_dict = {'index_column': X_train.columns, 'list_column': list_col}

# 데이터프레임 생성
df_feature = pd.DataFrame(data_dict)

In [22]:
df_feature = df_feature.sort_values(by = 'list_column', ascending = False)

In [25]:
fe_im = list(df_feature['index_column'])

In [26]:
n = 32
while n > 30 :
  X_f = df_oo[fe_im[:n]]
  from sklearn.model_selection import train_test_split
  X_train2, X_test2, y_train2, y_test2 = train_test_split(X_f, y_label, test_size = 0.2, random_state = 42)
  # 모델
  cat_model = CatBoostRegressor()
  # 학습
  cat_model.fit(X_train2, y_train2)
  # 예측
  lgbm_pred2 = cat_model.predict(X_test2)
  # 평가
  print( '{} 번째'.format(n))
  print(fe_im[n:])
  evaluation(y_test2, lgbm_pred2)
  n = n - 1

Learning rate set to 0.06345
0:	learn: 691.3754971	total: 6.14ms	remaining: 6.14s
1:	learn: 682.8520804	total: 10.9ms	remaining: 5.46s
2:	learn: 674.9211410	total: 16.2ms	remaining: 5.37s
3:	learn: 667.8686424	total: 22.4ms	remaining: 5.57s
4:	learn: 662.1868395	total: 26.1ms	remaining: 5.19s
5:	learn: 657.0908275	total: 29.3ms	remaining: 4.85s
6:	learn: 651.5651445	total: 34.3ms	remaining: 4.87s
7:	learn: 647.0975298	total: 39ms	remaining: 4.84s
8:	learn: 643.4482729	total: 44.9ms	remaining: 4.94s
9:	learn: 639.7967901	total: 51.5ms	remaining: 5.1s
10:	learn: 636.7211951	total: 56.3ms	remaining: 5.06s
11:	learn: 634.0749311	total: 60.9ms	remaining: 5.02s
12:	learn: 631.5422887	total: 65.7ms	remaining: 4.99s
13:	learn: 628.9720970	total: 70.6ms	remaining: 4.97s
14:	learn: 626.7655117	total: 75.6ms	remaining: 4.96s
15:	learn: 624.5015188	total: 80.7ms	remaining: 4.96s
16:	learn: 622.4479225	total: 85.8ms	remaining: 4.96s
17:	learn: 620.9228042	total: 90.7ms	remaining: 4.95s
18:	learn: 6